# Tutorial

This is a tutorial for MB-MVPA using task-fMRI data of Mixed-gamble task by Tom et al., 2007. 



### Import the MB-MVPA libarary.

Other libraries(nilean, keras, etc..) dosen't need to be imported.<br>
Because mb-mvpa has wrapping the libararies.<br>
You don't necessarily have to know fMRI libraries like nilearn and machine learning libraries like tensorflow.<br>
<b>MB-MVPA is all you need.</b>

Most of mb-mvpa are wrapping nilearn, tensorflow, Keras and etc., so warning can occur from that libraries.<br>
This page does not print warning because most of them are can be ignored.<br>
You don't need to remove the warning when you are actually using it.

In [1]:
from time import perf_counter

In [2]:
from mbmvpa.preprocessing.preprocess import DataPreprocessor

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/nilearn/glm/__init__.py:56: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  'It may change in any future release of Nilearn.', FutureWarning)


TODO: add original data download link

Data download from AWS S3, ~ <b>1GB</b> (would be under the "Mixed-gamble_task/example_data/").<br>

We provide a small subset (2 subjects) of original Tom's dataset (16 subjects). The fMRI images in the example is preprocessed by conventional fMRI preprocessing pipeline by using 
[*fmriprep*](https://fmriprep.org/en/stable/) v.20.1.0. Please refer to the [original](https://openneuro.org/datasets/ds000005/versions/00001) for more information.

In [3]:
#root = load_example_data("tom")
root = "/data2/project_modelbasedMVPA/ds000005"

### Preprocessing fMRI images and behavioral data

MB-MVPA requires primariliy preprocessed task-fMRI experiments data fromatted in conventional [BIDS format](https://bids-specification.readthedocs.io/en/stable/) 

It expects the following organized files. All the naming conventions used here conform with outputs from *fmriprep* v.20.1.0. by Poldrack lab.

The fMRI images are usually located here<br>
<i>{BIDS_ROOT}/derivatives/fmriprep/subject/session/run/func/*nii.gz</i><br>
And the behavior data are located here<br>
<i>{BIDS_ROOT}/subject/session/run/func/*.tsv</i>

In [4]:
s = perf_counter()

dm_model = 'ra_prospect'

def example_adjust(row):
    ## rename data in a row to the name which can match hbayesdm.ra_prospect requirements ##
    row["gamble"] = 1 if row["respcat"] == 1 else 0
    row["cert"] = 0
    return row

def example_filter(row):
    # include all trial data
    return True

def example_latent(row, param_dict):
    ## calculate subjectives utility for choosing Gamble over Safe option
    ## prospect theory with loss aversion and risk aversion is adopted
    modulation = (row["gain"] ** param_dict["rho"]) - (param_dict["lambda"] * (row["loss"] ** param_dict["rho"]))
    row["modulation"] = modulation
    return row


preprocessor = DataPreprocessor(bids_layout=root,
                               adjust_function=example_adjust,
                               filter_function=example_filter,
                               latent_function=example_latent,
                               dm_model=dm_model)
print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

elapsed time: 0.39 minutes


In [5]:
preprocessor.summary()

[  fMRIPrep  ] BIDS Layout: .../ds000005/derivatives/fmriprep | Subjects: 16 | Sessions: 0 | Runs: 48
[  MB-MVPA   ] BIDS Layout: ...PA/ds000005/derivatives/mbmvpa | Subjects: 16 | Sessions: 0 | Runs: 48


In [6]:
s = perf_counter()

preprocessor.preprocess(overwrite=False)

print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

Using cached StanModel: cached-ra_prospect-pystan_2.19.1.1.pkl



Model  = ra_prospect
Data   = <pandas.DataFrame object>

Details:
 # of chains                    = 4
 # of cores used                = 1
 # of MCMC samples (per chain)  = 4000
 # of burn-in samples           = 1000
 # of subjects                  = 16
 # of (max) trials per subject  = 256

Using cached StanModel: cached-ra_prospect-pystan_2.19.1.1.pkl


SystemError: <class 'stanfit4ra_prospect_39661c43d0c77275c6781f36a16d9609_6009202008874363709.PyStanHolder'> returned a result with an error set

In [ ]:
preprocessor.summary()

### Load data and shape check

In [3]:
from mbmvpa.data.loader import BIDSDataLoader

In [5]:
root = "/data2/project_modelbasedMVPA/ds000005"

In [7]:
s = perf_counter()

loader = BIDSDataLoader(layout=root)
X,y = loader.get_total_data(reconstruct=True)

print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

elapsed time: 0.24 minutes


In [ ]:
print("X", X.shape)
print("y", y.shape)

In [9]:
voxel_mask = loader.get_voxel_mask()

### Fitting MVPA models & Results

In [7]:
from mbmvpa.models.mvpa_cnn import CNN

In [14]:
MVPA_model = CNN(X=X,
                y=y,
                n_repeat=15,
                voxel_mask = voxel_mask)



In [ ]:
s = perf_counter()

coeffs = MVPA_model.run()

print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

[001] - val_loss: 0.1239
[002] - val_loss: 0.1248
[003] - val_loss: 0.1248
[004] - val_loss: 0.1226
[005] - val_loss: 0.1294
[006] - val_loss: 0.1255
[007] - val_loss: 0.1362
[008] - val_loss: 0.1389
[009] - val_loss: 0.1469
[010] - val_loss: 0.1423
[011] - val_loss: 0.1569
[012] - val_loss: 0.1501
[013] - val_loss: 0.1426
[014] - val_loss: 0.1583
[015] - val_loss: 0.1713
[016] - val_loss: 0.1502
[017] - val_loss: 0.1586
[018] - val_loss: 0.1559
[019] - val_loss: 0.1575
[020] - val_loss: 0.1682
[021] - val_loss: 0.1528
[022] - val_loss: 0.1587


In [ ]:
s = perf_counter()

sham_errors = MVPA_model.sham()

print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

In [ ]:
img = MVPA_model.image(save_path='.', task_name='example')